In [ ]:
from google.colab import files
files.upload()  #  kaggle.json dosyası

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json


In [ ]:
import zipfile

!kaggle competitions download -c obss-intern-competition-2025


zipfile.ZipFile("obss-intern-competition-2025.zip").extractall("caption_data")


 94% 934M/995M [00:00<00:00, 1.36GB/s]
100% 995M/995M [00:00<00:00, 1.37GB/s]


In [ ]:

!pip install transformers


In [ ]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
import torch
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"

processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16).to(device)



Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/882 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

In [ ]:
import pandas as pd

test_df = pd.read_csv("caption_data/test.csv")
test_df.head()


,image_id
0,100000
1,100001
2,100002
3,100003
4,100004


**bu kısım önmli**

In [ ]:
image_path = "/content/caption_data/test/test/100000.jpg"
raw_image = Image.open(image_path).convert("RGB")


inputs = processor(images=raw_image, return_tensors="pt").to(device, torch.float16)

out = model.generate(**inputs, max_new_tokens=30)
caption = processor.decode(out[0], skip_special_tokens=True)
print(caption)


In [ ]:
import os

# Test görsel klasöründe kaç tane görsel var?
len(os.listdir("/content/caption_data/test/test"))


3771

**çalışan submission**

In [ ]:
import pandas as pd
from PIL import Image
import os

test_df = pd.read_csv("/content/caption_data/test.csv")

results = []

for i, row in test_df.iterrows():
    image_id = row["image_id"]
    image_filename = f"{image_id}.jpg"
    image_path = f"/content/caption_data/test/test/{image_filename}"

    try:
        raw_image = Image.open(image_path).convert("RGB")
        inputs = processor(images=raw_image, return_tensors="pt").to(device, torch.float16)
        out = model.generate(**inputs, max_new_tokens=30, num_beams=5, early_stopping=True)
        caption = processor.decode(out[0], skip_special_tokens=True)
    except Exception as e:
        print(f"Hata: {image_filename} - {e}")
        caption = ""

    #image_id yerine image_filename
    results.append((image_filename, caption))

    if i % 419 == 0:
        print(f"{i}/{len(test_df)} görsel işlendi")

submission_df = pd.DataFrame(results, columns=["image_id", "caption"])
submission_df.to_csv("submission.csv", index=False)
print("✅ submission.csv başarıyla oluşturuldu.")

from google.colab import files
files.download("submission.csv")


0/3771 görsel işlendi
419/3771 görsel işlendi
838/3771 görsel işlendi
1257/3771 görsel işlendi
1676/3771 görsel işlendi
2095/3771 görsel işlendi
2514/3771 görsel işlendi
2933/3771 görsel işlendi
3352/3771 görsel işlendi
✅ submission.csv başarıyla oluşturuldu.


In [ ]:
from google.colab import files
files.download("submission.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
from datasets import Dataset
import os

df = pd.read_csv("/content/caption_data/train.csv")
df = df.dropna().reset_index(drop=True)

df["image_path"] = df["image_id"].apply(lambda x: f"/content/caption_data/train/train/{x}.jpg")

df = df[df["image_path"].apply(os.path.exists)]

dataset = Dataset.from_pandas(df[["image_path", "caption"]])


4. Görselleri Yükleyen Fonksiyon

In [ ]:
from PIL import Image

def preprocess(example):
    image = Image.open(example["image_path"]).convert("RGB")
    inputs = processor(images=image, text=example["caption"], return_tensors="pt", padding="max_length", truncation=True)
    example["pixel_values"] = inputs["pixel_values"][0]
    example["input_ids"] = inputs["input_ids"][0]
    example["attention_mask"] = inputs["attention_mask"][0]
    return example

dataset = dataset.map(preprocess)


Map:   0%|          | 0/21367 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


model eğitimi

In [ ]:
from transformers import TrainingArguments, Trainer
import os
os.environ["WANDB_DISABLED"] = "true"

from transformers import BlipProcessor, BlipForConditionalGeneration
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import pandas as pd
from datasets import Dataset
import os

df = pd.read_csv("/content/caption_data/train.csv")
df = df.dropna().reset_index(drop=True)
df["image_path"] = df["image_id"].apply(lambda x: f"/content/caption_data/train/train/{x}.jpg")
df = df[df["image_path"].apply(os.path.exists)]

dataset = Dataset.from_pandas(df[["image_path", "caption"]])


# **BLIP base**

In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

import pandas as pd
from datasets import Dataset
import os

df = pd.read_csv("/content/caption_data/train.csv")
df = df.dropna().reset_index(drop=True)
df["image_path"] = df["image_id"].apply(lambda x: f"/content/caption_data/train/train/{x}.jpg")
df = df[df["image_path"].apply(os.path.exists)]

dataset = Dataset.from_pandas(df[["image_path", "caption"]])

from PIL import Image


def preprocess(example):
    image = Image.open(example["image_path"]).convert("RGB")
    inputs = processor(images=image, text=example["caption"], padding="max_length", truncation=True, return_tensors="pt")
    example["pixel_values"] = inputs.pixel_values[0]
    example["input_ids"] = inputs.input_ids[0]
    example["attention_mask"] = inputs.attention_mask[0]
    example["labels"] = inputs.input_ids[0]
    return example
dataset = dataset.map(preprocess)

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./blip1-finetuned",
    per_device_train_batch_size=4,
    num_train_epochs=1,
    logging_steps=100,
    save_steps=500,
    save_total_limit=1,
    fp16=True,
    remove_unused_columns=False,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=processor,
)

trainer.train()


Map:   0%|          | 0/21367 [00:00<?, ? examples/s]

<ipython-input-6-10443736a83f>:47: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
100,3.888300
200,0.157500
300,0.138200
400,0.127800
500,0.124100
600,0.116800
700,0.110200
800,0.111200
900,0.108000
1000,0.108800


TrainOutput(global_step=5342, training_loss=0.16907504817958363, metrics={'train_runtime': 6536.2276, 'train_samples_per_second': 3.269, 'train_steps_per_second': 0.817, 'total_flos': 1.2679683177092088e+19, 'train_loss': 0.16907504817958363, 'epoch': 1.0})

In [ ]:
model.save_pretrained("/content/blip1-finetuned")
processor.save_pretrained("/content/blip1-finetuned")
import shutil

shutil.make_archive("/content/blip1-finetuned", 'zip', "/content/blip1-finetuned")
from google.colab import files

files.download("/content/blip1-finetuned.zip")


In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch
# !unzip /content/drive/MyDrive/blip1-finetuned.zip

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BlipForConditionalGeneration.from_pretrained("/content/blip1-finetuned-large").to(device)
processor = BlipProcessor.from_pretrained("/content/blip1-finetuned-large")


In [ ]:
def preprocess(example):
    image = Image.open(example["image_path"]).convert("RGB")
    inputs = processor(images=image, text=example["caption"], padding="max_length", truncation=True, return_tensors="pt")
    example["pixel_values"] = inputs.pixel_values[0]
    example["input_ids"] = inputs.input_ids[0]
    example["attention_mask"] = inputs.attention_mask[0]
    example["labels"] = inputs.input_ids[0]
    return example


In [ ]:
test_df = pd.read_csv("/content/caption_data/test.csv")
results = []

for i, row in test_df.iterrows():
    image_id = row["image_id"]
    image_path = f"/content/caption_data/test/test/{image_id}.jpg"

    try:
        image = Image.open(image_path).convert("RGB")
        inputs = processor(images=image, return_tensors="pt").to(device)
        out = model.generate(**inputs, max_new_tokens=30)
        caption = processor.tokenizer.decode(out[0], skip_special_tokens=True)
    except Exception as e:
        print(f"Hata: {image_id} - {e}")
        caption = ""

    results.append((f"{image_id}.jpg", caption))

    if i % 100 == 0:
        print(f"{i}/{len(test_df)} görsel işlendi")

submission_df = pd.DataFrame(results, columns=["image_id", "caption"])
submission_df.to_csv("submission.csv", index=False)
print("✅ submission.csv başarıyla oluşturuldu.")



**drive kodları**

In [ ]:
import zipfile
zip_path = "/content/drive/MyDrive/mt5-base-model.zip adlı dosyanın kopyası"
extract_path = "/content/blip1-finetuned"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [ ]:
import os
os.listdir(extract_path)


['content']

In [ ]:
import pandas as pd
from PIL import Image
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration

# Cihaz kontrolü
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Modelin yolunu verelim (Drive'dan çıkardığın klasör)
model_path = "/content/blip1-finetuned-large"

# Model ve tokenizer/processor yükleme
processor = BlipProcessor.from_pretrained(model_path)
model = BlipForConditionalGeneration.from_pretrained(model_path).to(device)

# Test CSV'yi yükle
test_df = pd.read_csv("/content/caption_data/test.csv")

results = []

# Caption oluşturma döngüsü
for i, row in test_df.iterrows():
    image_id = row["image_id"]
    image_path = f"/content/caption_data/test/test/{image_id}.jpg"

    try:
        image = Image.open(image_path).convert("RGB")
        inputs = processor(images=image, return_tensors="pt").to(device)
        output = model.generate(**inputs, max_new_tokens=30)
        caption = processor.tokenizer.decode(output[0], skip_special_tokens=True)
    except Exception as e:
        print(f"⚠️ Hata oluştu: {image_id}.jpg - {e}")
        caption = ""

    results.append((f"{image_id}.jpg", caption))

    if i % 100 == 0:
        print(f"🔄 {i}/{len(test_df)} görsel işlendi")

# DataFrame ve CSV kaydı
submission_df = pd.DataFrame(results, columns=["image_id", "caption"])
submission_df.to_csv("submission.csv", index=False)

print("✅ submission.csv başarıyla oluşturuldu.")


In [ ]:
from PIL import Image

image_path = "/content/drive/MyDrive/test_images/100001.jpg"
raw_image = Image.open(image_path).convert('RGB')

inputs = processor(raw_image, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")
out = model.generate(**inputs)
caption = processor.decode(out[0], skip_special_tokens=True)
print("Caption:", caption)


In [ ]:
import pandas as pd

# Mevcut submission dosyasını yükle
df = pd.read_csv("submission.csv")

# .jpg uzantısını kaldır
df["image_id"] = df["image_id"].str.replace(".jpg", "", regex=False)

# Yeni dosya olarak kaydet
df.to_csv("submission_fixed.csv", index=False)


# BLIP Large

In [ ]:

from transformers import BlipProcessor, BlipForConditionalGeneration
import torch
import pandas as pd
from datasets import Dataset
import os
from PIL import Image
from transformers import TrainingArguments, Trainer

# Cihaz ayarla
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Daha güçlü BLIP-Large modeli
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to(device)

# 2. Veri hazırlığı
df = pd.read_csv("/content/caption_data/train.csv")
df = df.dropna().reset_index(drop=True)
df["image_path"] = df["image_id"].apply(lambda x: f"/content/caption_data/train/train/{x}.jpg")
df = df[df["image_path"].apply(os.path.exists)]
dataset = Dataset.from_pandas(df[["image_path", "caption"]])

# 3. Preprocess fonksiyonu
def preprocess(example):
    image = Image.open(example["image_path"]).convert("RGB")
    # Prompt ekleyerek fine-tune et
    prompt = "Describe the image in detail: " + example["caption"]
    inputs = processor(images=image, text=prompt, padding="max_length", truncation=True, return_tensors="pt")
    example["pixel_values"] = inputs.pixel_values[0]
    example["input_ids"] = inputs.input_ids[0]
    example["attention_mask"] = inputs.attention_mask[0]
    example["labels"] = inputs.input_ids[0]
    return example

dataset = dataset.map(preprocess)

# 4. Eğitim ayarları
training_args = TrainingArguments(
    output_dir="./blip1-finetuned-large",
    per_device_train_batch_size=4,
    num_train_epochs=3,  # ✔ 3 epoch
    logging_steps=100,
    save_steps=500,
    save_total_limit=1,
    fp16=True,
    remove_unused_columns=False,
    report_to="none"
)

# 5. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=processor,
)

trainer.train()

# Not: Ğeitim bitince model kaydedilecek
model.save_pretrained("/content/blip1-finetuned-large")
processor.save_pretrained("/content/blip1-finetuned-large")


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

...

Map:   0%|          | 0/21367 [00:00<?, ? examples/s]

<ipython-input-4-5bcb1cb769dd>:52: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
100,3.613600
200,0.151700
300,0.136900
400,0.127400
500,0.125400
600,0.143200
700,0.114700
800,0.114300
900,0.111800
1000,0.113000


Step,Training Loss
100,3.613600
200,0.151700
300,0.136900
400,0.127400
500,0.125400
600,0.143200
700,0.114700
800,0.114300
900,0.111800
1000,0.113000


[]

In [ ]:
# ZIP'e çevir
!zip -r blip1-finetuned-large.zip blip1-finetuned-large

# İndirme bağlantısı
from google.colab import files
files.download("blip1-finetuned-large.zip")


updating: blip1-finetuned-large/ (stored 0%)
updating: blip1-finetuned-large/model.safetensors (deflated 7%)
updating: blip1-finetuned-large/preprocessor_config.json (deflated 49%)
updating: blip1-finetuned-large/generation_config.json (deflated 29%)
updating: blip1-finetuned-large/vocab.txt (deflated 53%)
updating: blip1-finetuned-large/tokenizer_config.json (deflated 74%)
updating: blip1-finetuned-large/config.json (deflated 66%)
updating: blip1-finetuned-large/tokenizer.json (deflated 71%)
updating: blip1-finetuned-large/checkpoint-16026/ (stored 0%)
updating: blip1-finetuned-large/checkpoint-16026/model.safetensors (deflated 7%)
updating: blip1-finetuned-large/checkpoint-16026/preprocessor_config.json (deflated 49%)
updating: blip1-finetuned-large/checkpoint-16026/scaler.pt (deflated 60%)
updating: blip1-finetuned-large/checkpoint-16026/rng_state.pth (deflated 25%)
updating: blip1-finetuned-large/checkpoint-16026/generation_config.json (deflated 29%)
updating: blip1-finetuned-large

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Google Drive'ı bağla
from google.colab import drive
drive.mount('/content/drive')

# Model klasörünü Drive'a kopyala
!cp -r /content/blip1-finetuned-large /content/drive/MyDrive/blip1-finetuned-large
print("✅ Model başarıyla Drive'a kopyalandı.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Model başarıyla Drive'a kopyalandı.


In [ ]:
# PaliGemma ile Image Captioning Fine-Tune
# Gereksinimler: JAX, Flax, optax, PaliGemma weights

!pip install -q -U jax jaxlib flax optax einops tensorflow datasets transformers sentence-transformers

import os
import json
import numpy as np
from PIL import Image
from datasets import load_dataset, Dataset
from transformers import AutoProcessor, AutoModelForVision2Seq
from sentence_transformers import SentenceTransformer, util
import torch

#  Cihaz kontrolü
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# En iyi (büyük) PaliGemma modeli
model_name = "google/paligemma-3b"
processor = AutoProcessor.from_pretrained(model_name)
model = AutoModelForVision2Seq.from_pretrained(model_name, torch_dtype=torch.float16).to(device)

# Eğitim verisini örnekle hazırlayalım
# Not: Gerçek kullanımda caption_data/train.csv kullanılmalı
data = [{
    "image_path": f"/content/caption_data/train/train/{i}.jpg",
    "caption": caption
} for i, caption in zip([150, 151], ["A cat sitting on a windowsill.", "A man riding a horse."])]

dataset = Dataset.from_list(data)

# Preprocess fonksiyonu
from torchvision import transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

def preprocess(example):
    image = Image.open(example["image_path"]).convert("RGB")
    pixel_values = processor(images=image, return_tensors="pt").pixel_values[0]
    inputs = processor(
        text=f"Describe the image in detail: {example['caption']}",
        images=image,
        return_tensors="pt",
        padding="max_length",
        truncation=True
    )
    example["pixel_values"] = pixel_values
    example["input_ids"] = inputs.input_ids[0]
    example["attention_mask"] = inputs.attention_mask[0]
    example["labels"] = inputs.input_ids[0]
    return example

processed_dataset = dataset.map(preprocess)

# Trainer yerine basit döngü (küçük veri için)
model.train()
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

for epoch in range(1):
    for example in processed_dataset:
        inputs = {
            "pixel_values": example["pixel_values"].unsqueeze(0).to(device),
            "input_ids": example["input_ids"].unsqueeze(0).to(device),
            "attention_mask": example["attention_mask"].unsqueeze(0).to(device),
            "labels": example["labels"].unsqueeze(0).to(device)
        }
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        print(f"Loss: {loss.item()}")

# Caption üretimi (beam search ve prompt)
model.eval()
image = Image.open("/content/caption_data/test/test/100000.jpg").convert("RGB")
inputs = processor(images=image, text="Describe the image in detail:", return_tensors="pt").to(device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=30,
        num_beams=7,
        early_stopping=True
    )

caption = processor.batch_decode(outputs, skip_special_tokens=True)[0]
print("Caption:", caption)

# ✅ FGD ve Cosine Similarity ile Değerlendirme
model_eval = SentenceTransformer("all-MiniLM-L6-v2")
ref = "A cat sitting on a windowsill."
pred = caption

emb1 = model_eval.encode(ref, convert_to_tensor=True)
emb2 = model_eval.encode(pred, convert_to_tensor=True)
sim = util.cos_sim(emb1, emb2).item()
print(f"Cosine Similarity: {sim:.4f}")


In [ ]:
import pandas as pd
from PIL import Image
import torch
from tqdm import tqdm
import os

test_df = pd.read_csv("/content/caption_data/test.csv")
results = []

for i, row in tqdm(test_df.iterrows(), total=len(test_df)):
    image_id = row["image_id"]
    image_filename = f"{image_id}.jpg"
    image_path = f"/content/caption_data/test/test/{image_filename}"

    try:
        raw_image = Image.open(image_path).convert("RGB")


        prompt = "Describe the image in detail:"

        inputs = processor(images=raw_image, text=prompt, return_tensors="pt").to(device)

        # Beam + temperature + repetition_penaltyle caption
        output = model.generate(
            **inputs,
            max_new_tokens=50,
            num_beams=7,
            early_stopping=True,
            repetition_penalty=1.2,
            temperature=1.0
        )
        caption = processor.decode(output[0], skip_special_tokens=True).strip()

    except Exception as e:
        print(f"⚠️ Hata: {image_filename} - {e}")
        caption = ""

    results.append((image_filename, caption))

submission_df = pd.DataFrame(results, columns=["image_id", "caption"])
submission_df.to_csv("submission.csv", index=False)
print("✅ Yeni ve geliştirilmiş submission.csv başarıyla oluşturuldu.")


100%|██████████| 3771/3771 [35:35<00:00,  1.77it/s]

✅ Yeni ve geliştirilmiş submission.csv başarıyla oluşturuldu.


In [ ]:
!pip install -q sentence-transformers scipy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from numpy import cov, trace, iscomplexobj
from scipy.linalg import sqrtm
from tqdm import tqdm

# GTE-small modelini yükle
model = SentenceTransformer("thenlper/gte-small")

def calculate_fgd(solution_embed: np.ndarray, submission_embed: np.ndarray) -> float:
    mu1, sigma1 = solution_embed.mean(axis=0), cov(solution_embed, rowvar=False)
    mu2, sigma2 = submission_embed.mean(axis=0), cov(submission_embed, rowvar=False)
    ssdiff = np.sum((mu1 - mu2) ** 2.0)
    covmean = sqrtm(sigma1.dot(sigma2))

    if iscomplexobj(covmean):
        covmean = covmean.real

    return float(ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean))

# submission.csv dosyan /content/submission_cleaned.csv
submission_df = pd.read_csv("submission.csv")

# caption_data/train.csv dosyasından 1000 gerçek açıklama örneği
ground_df = pd.read_csv("/content/caption_data/train.csv").dropna().reset_index(drop=True)

# İlk 1000 caption'ı al
ground_truth_captions = list(ground_df["caption"][:1000])
submission_captions = list(submission_df["caption"][:1000])

# Embed'leri hesapla
ground_embeddings = model.encode(ground_truth_captions, convert_to_numpy=True, show_progress_bar=True)
submission_embeddings = model.encode(submission_captions, convert_to_numpy=True, show_progress_bar=True)

fgd_score = calculate_fgd(ground_embeddings, submission_embeddings)
print(f"🔍 Tahmini FGD Skoru: {fgd_score:.4f}")


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

🔍 Tahmini FGD Skoru: 0.0244


submissionu temizler

In [ ]:
import pandas as pd

# Hatalı dosyayı yükle
df = pd.read_csv("/content/submission_v2-2.csv")

# Caption'lardaki prompt kısmını temizle
df["caption"] = df["caption"].str.replace("(?i)^Caption this image in detail\\s*[:\\-]?\\s*", "", regex=True).str.strip()

# Yeni dosya olarak kaydet
df.to_csv("submission_cleaned.csv", index=False)
print("✅ Prompt temizlenmiş yeni submission.csv hazırlandı.")


✅ Prompt temizlenmiş yeni submission.csv hazırlandı.


In [ ]:
import pandas as pd

# Yanlış dosyayı yükle
df = pd.read_csv("/content/submission_cleaned.csv")
# .jpg uzantısını kaldır
df["image_id"] = df["image_id"].str.replace(".jpg", "", regex=False)

# Yeni dosyaya kaydet
df.to_csv("submission_fixed.csv", index=False)
print("✅ .jpg uzantısı kaldırıldı, submission_fixed.csv hazır.")


✅ .jpg uzantısı kaldırıldı, submission_fixed.csv hazır.


# **RAG DENEMESİ**
BLIP modelinin fine-tuned edilmiş versiyonu yüklenir ve her bir test


görseli için base açıklama üretilir. Bu açıklama, SentenceTransformer ile vektöre dönüştürülür ve eğitim verisindeki corpusla karşılaştırılarak anlamca en yakın 3 açıklama bulunur. Bu benzer açıklamalar, FLAN-T5 dil modeli kullanılarak temel açıklamayı refine etmek için prompt'a dahil edilir. T5 modeline verilen prompt, açıklamanın anlamını değiştirmeden daha net ve ayrıntılı hale getirilmesini sağlar.

In [ ]:
# 📦 Kurulumlar
!pip install -q sentence-transformers transformers

# 📚 Gerekli kütüphaneler
import pandas as pd
import torch
from PIL import Image
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
from transformers import BlipProcessor, BlipForConditionalGeneration, AutoTokenizer, AutoModelForSeq2SeqLM

# Cihaz ayarı
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#  BLIP (finetuned) modelini yükle
blip_path = "/content/blip1-finetuned"  # kendi model yolunu yaz
processor = BlipProcessor.from_pretrained(blip_path)
blip_model = BlipForConditionalGeneration.from_pretrained(blip_path).to(device)

#  Caption embedding modeli (GTE-small)
embedder = SentenceTransformer("thenlper/gte-small")

#  FLAN-T5 (caption refinement için)
t5_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
t5_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base").to(device)

#  train.csv'den caption corpus'u
train_df = pd.read_csv("/content/caption_data/train.csv").dropna().reset_index(drop=True)
corpus_captions = train_df["caption"].tolist()[:1000]  # istersen artırabilirsin
corpus_embeddings = embedder.encode(corpus_captions, convert_to_tensor=True)

#  test.csv'yi yükle
test_df = pd.read_csv("/content/caption_data/test.csv")
results = []

#  Tüm test görselleri için caption oluştur + refine et
for i, row in tqdm(test_df.iterrows(), total=len(test_df)):
    image_id = row["image_id"]
    image_path = f"/content/caption_data/test/test/{image_id}.jpg"

    try:
        raw_image = Image.open(image_path).convert("RGB")

        # 1. BLIP ile caption üret
        inputs = processor(images=raw_image, return_tensors="pt").to(device)
        out = blip_model.generate(
            **inputs,
            max_new_tokens=50,
            num_beams=7,
            early_stopping=True,
            repetition_penalty=1.2,
            temperature=0.9
        )
        base_caption = processor.decode(out[0], skip_special_tokens=True).strip()

        # 2. En benzer 3 caption'ı train setinden bul
        query_embedding = embedder.encode(base_caption, convert_to_tensor=True)
        hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=3)[0]
        similar_caps = [corpus_captions[hit["corpus_id"]] for hit in hits]

        # 3. Caption'ı T5 ile refine et
        prompt = f"""Base caption: "{base_caption}"\nSimilar captions: {similar_caps}\n\nImprove the base caption slightly for clarity and detail. Do not change its original meaning or structure."""
        inputs_t5 = t5_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)
        output_t5 = t5_model.generate(**inputs_t5, max_new_tokens=50)
        final_caption = t5_tokenizer.decode(output_t5[0], skip_special_tokens=True).strip()

        # 2. En benzer 3 caption'ı train setinden bul
        query_embedding = embedder.encode(base_caption, convert_to_tensor=True)
        hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=3)[0]
        similar_caps = [corpus_captions[hit["corpus_id"]] for hit in hits]

        # 3. Caption'ı T5 ile refine et (GÜNCELLENMİŞ)
        similar_text = "\n".join(similar_caps)
        prompt = f"""Base caption: "{base_caption}"\nSimilar captions:\n{similar_text}\n\nImprove the base caption slightly for clarity and detail. Do not change its original meaning or structure."""
        inputs_t5 = t5_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)
        output_t5 = t5_model.generate(**inputs_t5, max_new_tokens=50)
        final_caption = t5_tokenizer.decode(output_t5[0], skip_special_tokens=True).strip()
        final_caption = final_caption.replace("[", "").replace("]", "").replace("'", "").strip()



    except Exception as e:
        print(f"Hata: {image_id} - {e}")
        final_caption = ""

    results.append((image_id, final_caption))

#  submission.csv oluştur
submission_df = pd.DataFrame(results, columns=["image_id", "caption"])
submission_df.to_csv("submission.csv", index=False)
print(" Final caption'larla submission.csv dosyası oluşturuldu.")


  0%|          | 0/3771 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
100%|█████████▉| 3756/3771 [1:13:46<00:15,  1.05s/it]

In [ ]:
from google.colab import files
# files.download("submission.csv")
files.download("/content/basemt5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

FGD

In [ ]:
import pandas as pd
import numpy as np
from numpy import cov, trace, iscomplexobj
from scipy.linalg import sqrtm
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer("thenlper/gte-small")

gt_df = pd.read_csv("ground_truth.csv")         # image_id, caption (doğru açıklama)
sub_df = pd.read_csv("submission.csv")          # image_id, caption (tahminin)

# image_id üzerinden eşleştir
merged_df = pd.merge(gt_df, sub_df, on="image_id", suffixes=("_gt", "_pred"))
gt_captions = merged_df["caption_gt"].tolist()
pred_captions = merged_df["caption_pred"].tolist()

# ✅ Embed'leri hesapla
gt_embed = embedder.encode(gt_captions, convert_to_tensor=False)
pred_embed = embedder.encode(pred_captions, convert_to_tensor=False)

# ✅ FGD hesaplayıcı
def calculate_fgd(gt_embed: np.ndarray, pred_embed: np.ndarray) -> float:
    fgd_list = []
    for sol_emb, sub_emb in zip(gt_embed, pred_embed):
        e1 = np.vstack([sol_emb, sol_emb])
        e2 = np.vstack([sub_emb, sub_emb])

        mu1, sigma1 = e1.mean(axis=0), cov(e1, rowvar=False)
        mu2, sigma2 = e2.mean(axis=0), cov(e2, rowvar=False)

        ssdiff = np.sum((mu1 - mu2) ** 2.0)
        covmean = sqrtm(sigma1.dot(sigma2))

        if iscomplexobj(covmean):
            covmean = covmean.real

        fgd = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
        fgd_list.append(fgd)
    return float(np.mean(fgd_list))

# ✅ FGD skorunu yazdır
fgd_score = calculate_fgd(gt_embed, pred_embed)
print(f"📊 Your FGD Score: {fgd_score:.4f} (Lower is better)")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

FileNotFoundError: [Errno 2] No such file or directory: 'ground_truth.csv'

PALİGEMMA


In [ ]:

import torch
from transformers import (
    AutoProcessor,
    PaliGemmaForConditionalGeneration,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from datasets import Dataset
from PIL import Image
import pandas as pd
import os

model_id = "google/paligemma2-3b-mix-224"
model = PaliGemmaForConditionalGeneration.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")
processor = AutoProcessor.from_pretrained(model_id)

max_length = 128

df = pd.read_csv("/content/caption_data/train.csv").dropna()
df["image_path"] = df["image_id"].apply(lambda x: f"/content/caption_data/train/train/{x}.jpg")
df = df[df["image_path"].apply(os.path.exists)]
dataset = Dataset.from_pandas(df[["image_path", "caption"]])

def preprocess(example):
    image = Image.open(example["image_path"]).convert("RGB")
    prompt = f"<image> {example['caption']}"
    processed = processor(
        images=image,
        text=prompt,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )
    return {
        "pixel_values": processed["pixel_values"].squeeze(0),  # [1, C, H, W] → [C, H, W]
        "input_ids": processed["input_ids"].squeeze(0),
        "attention_mask": processed["attention_mask"].squeeze(0),
        "labels": processed["input_ids"].squeeze(0)
    }

processed_examples = [preprocess(row) for row in dataset]
final_dataset = Dataset.from_list(processed_examples)

data_collator = DataCollatorWithPadding(tokenizer=processor.tokenizer, pad_to_multiple_of=8)

training_args = TrainingArguments(
    output_dir="./paligemma-finetuned",
    per_device_train_batch_size=2,
    num_train_epochs=1,
    logging_steps=50,
    save_steps=500,
    save_total_limit=1,
    remove_unused_columns=False,
    report_to="none",
    fp16=True,
    dataloader_pin_memory=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=final_dataset,
    tokenizer=processor,
    data_collator=data_collator,
)

trainer.train()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# # ✅ Gerekli kütüphaneleri yükle
# !pip install -q transformers datasets accelerate

import os
import torch
import pandas as pd
from PIL import Image
from datasets import Dataset
from transformers import AutoProcessor, AutoModelForImageTextToText, TrainingArguments, Trainer

# ✅ Model ID
model_id = "google/paligemma2-3b-mix-224"

# ✅ Model ve Processor yükleniyor
processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForImageTextToText.from_pretrained(model_id, torch_dtype=torch.bfloat16)

# ✅ Eğitim verisi yükle
df = pd.read_csv("/content/caption_data/train.csv")
df["image_path"] = df["image_id"].apply(lambda x: f"/content/caption_data/train/train/{x}.jpg")
df = df[df["image_path"].apply(os.path.exists)]

# ✅ Dataset formatına çevir
dataset = Dataset.from_pandas(df[["image_path", "caption"]])

# ✅ Preprocessing fonksiyonu
from PIL import Image

def preprocess(example):
    image = Image.open(example["image_path"]).convert("RGB")

    # Metnin başına 1 tane <image> token eklenmeli
    prompt = "<image> Describe this image."

    # Tokenize et
    inputs = processor(
        images=image,
        text=prompt,
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

    labels = processor.tokenizer(
        example["caption"],
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt"
    ).input_ids[0]

    return {
        "pixel_values": inputs["pixel_values"][0],
        "input_ids": inputs["input_ids"][0],
        "attention_mask": inputs["attention_mask"][0],
        "labels": labels
    }




# ✅ Dataset preprocess ediliyor
dataset = dataset.map(preprocess, remove_columns=["image_path", "caption"])

# ✅ Eğitim ayarları
training_args = TrainingArguments(
    output_dir="./gemma-captioning",
    per_device_train_batch_size=2,
    num_train_epochs=2,
    save_steps=500,
    save_total_limit=1,
    logging_steps=50,
    remove_unused_columns=False,
    fp16=True,
    report_to="none"
)

# ✅ Trainer başlat
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=processor.tokenizer,

)

# ✅ Eğitimi başlat
trainer.train()

# ✅ Test verisi yükle
test_df = pd.read_csv("/content/caption_data/test.csv")
test_df["image_path"] = test_df["image_id"].apply(lambda x: f"/content/caption_data/test/test/{x}.jpg")
test_df = test_df[test_df["image_path"].apply(os.path.exists)]

# ✅ Caption üretim fonksiyonu
def generate_caption(image_path):
    image = Image.open(image_path).convert("RGB").resize((896, 896))
    inputs = processor(images=image, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=64)
    caption = processor.decode(outputs[0], skip_special_tokens=True)
    return caption

# ✅ Tüm test verisi için caption üret
captions = []
for path in test_df["image_path"]:
    try:
        caption = generate_caption(path)
    except Exception as e:
        caption = "Error"
    captions.append(caption)

# ✅ Submission dosyasını oluştur
submission_df = pd.DataFrame({
    "image_id": test_df["image_id"],
    "caption": captions
})

submission_df.to_csv("submission.csv", index=False)
print("✅ submission.csv başarıyla oluşturuldu.")


In [ ]:
from tqdm import tqdm
import pandas as pd
from PIL import Image
import torch
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration

model_path = "/content/paligemma-finetuned"
model = PaliGemmaForConditionalGeneration.from_pretrained(model_path, torch_dtype=torch.bfloat16).to("cuda")
processor = AutoProcessor.from_pretrained(model_path)

test_df = pd.read_csv("/content/caption_data/test.csv")
results = []

for i, row in tqdm(test_df.iterrows(), total=len(test_df)):
    image_id_jpg = row["image_id"]
    image_id = image_id_jpg.replace(".jpg", "")
    image_path = f"/content/caption_data/test/test/{image_id}.jpg"

    try:
        image = Image.open(image_path).convert("RGB")
        inputs = processor(
            images=image,
            text="<image> Describe this image.",
            return_tensors="pt",
            padding="max_length",
            truncation=True
        ).to("cuda")

        output_ids = model.generate(
            **inputs,
            max_new_tokens=50,
            do_sample=False,
            num_beams=5
        )

        caption = processor.decode(output_ids[0], skip_special_tokens=True).strip()
    except Exception as e:
        print(f"Hata: {image_id} - {e}")
        caption = ""

    results.append((image_id, caption))


submission_df = pd.DataFrame(results, columns=["image_id", "caption"])
submission_df.to_csv("submission.csv", index=False)
print("✅ submission.csv başarıyla oluşturuldu.")


In [ ]:
# # ✅ Gerekli kütüphaneleri yükle
# !pip install -q transformers datasets accelerate

import os
import torch
import pandas as pd
from PIL import Image
from datasets import Dataset
from transformers import AutoProcessor, AutoModelForImageTextToText, TrainingArguments, Trainer

# ✅ Model ID
model_id = "google/gemma-3-4b-it-qat-q4_0-unquantized"

# ✅ Model ve Processor yükleniyor
processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForImageTextToText.from_pretrained(model_id, torch_dtype=torch.bfloat16)

# ✅ Eğitim verisi yükle
df = pd.read_csv("/content/caption_data/train.csv")
df["image_path"] = df["image_id"].apply(lambda x: f"/content/caption_data/train/train/{x}.jpg")
df = df[df["image_path"].apply(os.path.exists)]

# ✅ Dataset formatına çevir
dataset = Dataset.from_pandas(df[["image_path", "caption"]])

# ✅ Preprocessing fonksiyonu
from PIL import Image

def preprocess(example):
    from PIL import Image
    import torch

    image = Image.open(example["image_path"]).convert("RGB")

    # ✅ image token ID
    image_token_id = processor.tokenizer.convert_tokens_to_ids(processor.tokenizer.image_token)

    # ✅ metni token'la birleştir
    text_ids = processor.tokenizer(" Describe this image.", return_tensors="pt", add_special_tokens=False).input_ids[0]
    input_ids = torch.cat([torch.tensor([image_token_id]), text_ids], dim=0)
    attention_mask = torch.ones_like(input_ids)

    # ✅ sadece görseli işlemek için image_processor kullan
    pixel_values = processor.image_processor(images=image, return_tensors="pt")["pixel_values"][0]

    # ✅ hedef caption
    labels = processor.tokenizer(example["caption"], return_tensors="pt").input_ids[0]

    return {
        "pixel_values": pixel_values,
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }


# ✅ Dataset preprocess ediliyor
dataset = dataset.map(preprocess, remove_columns=["image_path", "caption"])

# ✅ Eğitim ayarları
training_args = TrainingArguments(
    output_dir="./gemma-captioning",
    per_device_train_batch_size=2,
    num_train_epochs=2,
    save_steps=500,
    save_total_limit=1,
    logging_steps=50,
    remove_unused_columns=False,
    fp16=True,
    report_to="none"
)

# ✅ Trainer başlat
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=processor,
)

# ✅ Eğitimi başlat
trainer.train()

# ✅ Test verisi yükle
test_df = pd.read_csv("/content/caption_data/test.csv")
test_df["image_path"] = test_df["image_id"].apply(lambda x: f"/content/caption_data/test/test/{x}.jpg")
test_df = test_df[test_df["image_path"].apply(os.path.exists)]

# ✅ Caption üretim fonksiyonu
def generate_caption(image_path):
    image = Image.open(image_path).convert("RGB").resize((896, 896))
    inputs = processor(images=image, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=64)
    caption = processor.decode(outputs[0], skip_special_tokens=True)
    return caption

# ✅ Tüm test verisi için caption üret
captions = []
for path in test_df["image_path"]:
    try:
        caption = generate_caption(path)
    except Exception as e:
        caption = "Error"
    captions.append(caption)

# ✅ Submission dosyasını oluştur
submission_df = pd.DataFrame({
    "image_id": test_df["image_id"],
    "caption": captions
})

submission_df.to_csv("submission.csv", index=False)
print("✅ submission.csv başarıyla oluşturuldu.")


In [ ]:
# Tüm belleği sıfırlamak için
import os
os.kill(os.getpid(), 9)


In [ ]:
print(processor.tokenizer.special_tokens_map)


In [ ]:
image_token_id = processor.tokenizer.convert_tokens_to_ids(processor.tokenizer.image_token)
print("Image token ID:", image_token_id)

prompt_ids = processor.tokenizer(prompt, return_tensors="pt").input_ids[0]
print("Prompt token IDs:", prompt_ids.tolist())

print("Image token ID is in prompt:", image_token_id in prompt_ids)


Image token ID: 262144
Prompt token IDs: [2, 262144, 54234, 672, 2471, 236761]
Image token ID is in prompt: True


In [ ]:
# Gerekli Kurulumlar
!pip install -q transformers datasets accelerate bitsandbytes

# Kütüphaneler
import os
import torch
import pandas as pd
from PIL import Image
from datasets import Dataset
from transformers import (
    AutoProcessor,
    PaliGemmaForConditionalGeneration,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq
)

# Model ve processor
model_id = "google/paligemma-3b-ft-textcaps-448"

processor = AutoProcessor.from_pretrained(model_id)
model = PaliGemmaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

# Eğitim verisi
df = pd.read_csv("/content/caption_data/train.csv")
df["image_path"] = df["image_id"].apply(lambda x: f"/content/caption_data/train/train/{x}.jpg")
df = df[df["image_path"].apply(os.path.exists)]
dataset = Dataset.from_pandas(df[["image_path", "caption"]])

# Preprocessing fonksiyonu
def preprocess(example):
    image = Image.open(example["image_path"]).convert("RGB")
    prompt = "<image> Describe this image."  # PaliGemma'da önerilen format

    processed = processor(
        images=image,
        text=prompt,
        text_target=example["caption"],  # HuggingFace 4.38+ ile uyumlu
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=128
    )

    # pad token'larını ignore et
    labels = processed["labels"][0]
    labels[labels == processor.tokenizer.pad_token_id] = -100

    return {
        "pixel_values": processed["pixel_values"][0],
        "input_ids": processed["input_ids"][0],
        "attention_mask": processed["attention_mask"][0],
        "labels": labels,
    }


# Veriyi işle
dataset = dataset.map(preprocess, remove_columns=["image_path", "caption"])

# Data collator
data_collator = DataCollatorForSeq2Seq(processor.tokenizer, model=model, pad_to_multiple_of=8)

# Eğitim ayarları
training_args = TrainingArguments(
    output_dir="./paligemma-textcaps-obss",
    per_device_train_batch_size=2,
    num_train_epochs=2,
    logging_steps=50,
    save_steps=500,
    save_total_limit=1,
    fp16=True,
    report_to="none",
    remove_unused_columns=False
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=processor,
    data_collator=data_collator,
)

# Eğitimi başlat
trainer.train()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/21367 [00:00<?, ? examples/s]

<ipython-input-1-4289e168ff5f>:81: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


RuntimeError: "_amp_foreach_non_finite_check_and_unscale_cuda" not implemented for 'BFloat16'

In [ ]:
from google.colab import files
files.download("submission.csv")

In [ ]:
# Google Drive'ı bağla
from google.colab import drive
drive.mount('/content/drive')

# Model klasörünü Drive'a kopyala
!cp -r /content/paligemma-textcaps-obss /content/drive/MyDrive/paligemma-finetuned-capt
print("✅ Model başarıyla Drive'a kopyalandı.")

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `obss2` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate wh

In [ ]:
!pip install -q transformers accelerate datasets

import os
import pandas as pd
from PIL import Image
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration
from torch.optim import AdamW


# Ayarlar
model_id = "google/paligemma-3b-pt-224"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32
batch_size = 2
learning_rate = 1e-5
num_epochs = 2

# Dataset sınıfı
class MyCaptionDataset(Dataset):
    def __init__(self, csv_path):
        df = pd.read_csv(csv_path)
        self.image_paths = df["image_id"].apply(lambda x: f"/content/caption_data/train/train/{x}.jpg").tolist()
        self.captions = df["caption"].tolist()

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB").resize((224, 224))
        return {
            "image": image,
            "caption": self.captions[idx]
        }

    def __len__(self):
        return len(self.image_paths)

# Dataset'i yükle ve böl
full_dataset = MyCaptionDataset("/content/caption_data/train.csv")
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Processor ve model
processor = AutoProcessor.from_pretrained(model_id)
model = PaliGemmaForConditionalGeneration.from_pretrained(model_id).to(device=device, dtype=dtype)

def collate_fn(batch):
    images = [item["image"] for item in batch]
    prompts = ["<image> :" for _ in batch]  # ✅ önemli düzeltme
    suffixes = [item["caption"] for item in batch]

    inputs = processor(
        images=images,
        text=prompts,
        suffix=suffixes,
        return_tensors="pt",
        padding="longest",
    ).to(device)

    return inputs


# DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Optimizer
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Eğitim döngüsü
model.train()
for epoch in range(num_epochs):
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {total_loss/len(train_loader):.4f}")

# Modeli kaydet
model.save_pretrained("paligemma-ft2")
processor.save_pretrained("paligemma-ft2")
print("✅ Eğitim tamamlandı ve model kaydedildi.")


preprocessor_config.json:   0%|          | 0.00/699 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Epoch 1/2: 100%|██████████| 8547/8547 [37:30<00:00,  3.80it/s]


Epoch 1 Loss: 1.2892


Epoch 2/2: 100%|██████████| 8547/8547 [37:31<00:00,  3.80it/s]


Epoch 2 Loss: 1.1017
✅ Eğitim tamamlandı ve model kaydedildi.


In [ ]:
# Google Drive'ı bağla
from google.colab import drive
drive.mount('/content/drive')

# Model klasörünü Drive'a kopyala
!cp -r /content/paligemma-ft2 /content/drive/MyDrive/paligemma-finetuned-capt
print("✅ Model başarıyla Drive'a kopyalandı.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Model başarıyla Drive'a kopyalandı.


In [ ]:
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration
from PIL import Image
import torch

model_path = "/content/drive/MyDrive/paligemma-finetuned-capt/paligemma-ft2"  # veya Drive yolun
model = PaliGemmaForConditionalGeneration.from_pretrained(model_path).to("cuda")
processor = AutoProcessor.from_pretrained(model_path)

image = Image.open("/content/caption_data/test/test/100000.jpg").convert("RGB").resize((224, 224))
prompt = "<image> Caption the image:"

inputs = processor(images=image, text=prompt, return_tensors="pt").to("cuda")
output = model.generate(**inputs, max_new_tokens=50)
caption = processor.batch_decode(output, skip_special_tokens=True)[0]

print("🖼️ Caption:", caption)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.96 GiB. GPU 0 has a total capacity of 14.74 GiB of which 1.79 GiB is free. Process 65824 has 12.95 GiB memory in use. Of the allocated memory 12.46 GiB is allocated by PyTorch, and 371.37 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import pandas as pd
from tqdm import tqdm
from PIL import Image
import torch
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration
import os

#  Modeli yükle /content/drive/MyDrive/paligemma-ft2
model_path = "/content/paligemma-ft-v2-2"  # V1 için
model = PaliGemmaForConditionalGeneration.from_pretrained(model_path).to("cuda")
processor = AutoProcessor.from_pretrained(model_path)

# Test verisi yükle
test_df = pd.read_csv("/content/caption_data/test.csv")  # test.csv: image_id içermeli
test_dir = "/content/caption_data/test/test"

# Caption'ları üret
captions = []

for image_id in tqdm(test_df["image_id"]):
    image_path = os.path.join(test_dir, f"{image_id}.jpg")
    image = Image.open(image_path).convert("RGB").resize((224, 224))
    #    prompts = ["<image> Describe this image in detail, focusing on objects, setting, and visible actions:" for _ in batch]  # ✅ önemli düzeltme

    prompt = "<image> Caption this image in detail:"
    inputs = processor(images=image, text=prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=64)
        caption = processor.batch_decode(output, skip_special_tokens=True)[0]
        caption = caption.replace(prompt, "").strip()  # promptu sil

    captions.append(caption)

# CSV olarak kaydet
submission = pd.DataFrame({"image_id": test_df["image_id"], "caption": captions})
submission.to_csv("submission_v2-2.csv", index=False)
print("✅ Submission oluşturuldu: submission_v2-2.csv")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 3771/3771 [41:31<00:00,  1.51it/s]

✅ Submission oluşturuldu: submission_v2-2.csv


In [ ]:
from google.colab import files
files.download("submission_v2.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install -q transformers accelerate datasets

import os
import pandas as pd
from PIL import Image
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration
from torch.optim import AdamW

# Ayarlar
# model_id = "google/paligemma-3b-pt-224"
model_path = "/content/paligemma-ft-v2"
output_dir = "/content/paligemma-ft-v2-2"
drive_output_dir = "/content/drive/MyDrive/paligemma-finetuned-v2-2"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32
batch_size = 2
learning_rate = 5e-6
num_epochs = 2

# Dataset sınıfı
class MyCaptionDataset(Dataset):
    def __init__(self, csv_path):
        df = pd.read_csv(csv_path)
        self.image_paths = df["image_id"].apply(lambda x: f"/content/caption_data/train/train/{x}.jpg").tolist()
        self.captions = df["caption"].tolist()

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB").resize((224, 224))
        return {
            "image": image,
            "caption": self.captions[idx]
        }

    def __len__(self):
        return len(self.image_paths)

# Dataset'i yükle ve böl
full_dataset = MyCaptionDataset("/content/caption_data/train.csv")
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Processor ve model

processor = AutoProcessor.from_pretrained(model_path)
model = PaliGemmaForConditionalGeneration.from_pretrained(model_path).to(device=device, dtype=dtype)
# Collate fonksiyonu
def collate_fn(batch):
    images = [item["image"] for item in batch]
    prompts = ["<image> Caption this image in detail:" for _ in batch]
    suffixes = [item["caption"] for item in batch]

    inputs = processor(
        images=images,
        text=prompts,
        suffix=suffixes,
        return_tensors="pt",
        padding="longest",
    ).to(device)

    return inputs

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Optimizer
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Eğitim döngüsü
model.train()
for epoch in range(num_epochs):
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {total_loss/len(train_loader):.4f}")

# Modeli kaydet
model.save_pretrained(output_dir)
processor.save_pretrained(output_dir)
print(f"✅ Eğitim tamamlandı ve model '{output_dir}' klasörüne kaydedildi.")

# Google Drive'a yedekle
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!cp -r {output_dir} {drive_output_dir}
print(f"✅ Model başarıyla Drive'a yedeklendi: {drive_output_dir}")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1/2: 100%|██████████| 8547/8547 [38:33<00:00,  3.70it/s]


Epoch 1 Loss: 1.0529


Epoch 2/2: 100%|██████████| 8547/8547 [38:33<00:00,  3.69it/s]


Epoch 2 Loss: 1.0219
✅ Eğitim tamamlandı ve model '/content/paligemma-ft-v2-2' klasörüne kaydedildi.
Mounted at /content/drive
✅ Model başarıyla Drive'a yedeklendi: /content/drive/MyDrive/paligemma-finetuned-v2-2


In [ ]:
# Tüm belleği sıfırlamak için
import os
os.kill(os.getpid(), 9)


In [ ]:
google/paligemma-3b-pt-224

In [ ]:
!pip install -q transformers accelerate datasets

import os
import pandas as pd
from PIL import Image
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration
from torch.optim import AdamW


model_id = "google/paligemma-3b-pt-224"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32
batch_size = 2
learning_rate = 5e-6
num_epochs = 3

# Dataset sınıfı
class MyCaptionDataset(Dataset):
    def __init__(self, csv_path):
        df = pd.read_csv(csv_path)
        self.image_paths = df["image_id"].apply(lambda x: f"/content/caption_data/train/train/{x}.jpg").tolist()
        self.captions = df["caption"].tolist()

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB").resize((224, 224))
        return {
            "image": image,
            "caption": self.captions[idx]
        }

    def __len__(self):
        return len(self.image_paths)

# Dataset yükle ve böl
full_dataset = MyCaptionDataset("/content/caption_data/train.csv")
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Processor ve model
processor = AutoProcessor.from_pretrained(model_id)
model = PaliGemmaForConditionalGeneration.from_pretrained(model_id).to(device=device, dtype=dtype)

def collate_fn(batch):
    images = [item["image"] for item in batch]
    prompts = ["<image> Caption the image." for _ in batch]  # ✅ önemli düzeltme
    suffixes = [item["caption"] for item in batch]

    inputs = processor(
        images=images,
        text=prompts,
        suffix=suffixes,
        return_tensors="pt",
        padding="longest",
    ).to(device)

    return inputs


# DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Optimizer
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Eğitim döngüsü
model.train()
for epoch in range(num_epochs):
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {total_loss/len(train_loader):.4f}")

# Modeli kaydet
model.save_pretrained("paligemma-ft2")
processor.save_pretrained("paligemma-ft2")
print("Eğitim tamamlandı ve model kaydedildi.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.0 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/699 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Epoch 1/3:  17%|█▋        | 1477/8547 [06:39<31:51,  3.70it/s]


KeyboardInterrupt: 

In [ ]:
# Google Drive'ı bağla
from google.colab import drive
drive.mount('/content/drive')

# Model klasörünü Drive'a kopyala
!cp -r /content/paligemma-ft2 /content/drive/MyDrive/paligemma-ft2
print("✅ Model başarıyla Drive'a kopyalandı.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Model başarıyla Drive'a kopyalandı.
